<a href="https://colab.research.google.com/github/dorianxiao/DLexp/blob/master/Exp1%3A%E6%89%8B%E5%86%99%E6%95%B0%E5%AD%97%E8%AF%86%E5%88%AB/MNIST_CNN_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 初始化

In [0]:
# 导入相关库

from __future__ import absolute_import, division, print_function

import gzip
import os
import sys
import time

import numpy
from six.moves import urllib
from six.moves import xrange
import tensorflow as tf

In [0]:
# 定义全局变量

IMAGE_SIZE = 28
NUM_CHANNELS = 1
PIXEL_DEPTH = 255
NUM_LABELS = 10
VALIDATION_SIZE = 5000    # 验证集大小
SEED = 66478
BATCH_SIZE = 64
NUM_EPOCHS = 10
EVAL_BATCH_SIZE = 64
EVAL_FREQUENCY = 100

# 2. 下载数据

### 挂载到Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def download(filename):
  filepath = os.path.join(WORK_DIRECTORY, filename)
  
  # 如果没有下载
  if not tf.gfile.Exists(filepath):
    filepath, _ = urllib.request.urlretrieve(SOURCE_URL + filename, filepath)
    with tf.gfile.GFile(filepath) as f:
      size = f.size()
    print('成功下载 ', filename, size, 'bytes.')
  else:
    print('目录中已存在 ', filename)
  return filepath

In [0]:
# 指定下载目录

SOURCE_URL = 'http://yann.lecun.com/exdb/mnist/'
WORK_DIRECTORY = r'/content/gdrive/My Drive/mylab'

In [6]:
# 开始下载

train_data_filename = download('train-images-idx3-ubyte.gz')
train_labels_filename = download('train-labels-idx1-ubyte.gz')
test_data_filename = download('t10k-images-idx3-ubyte.gz')
test_labels_filename = download('t10k-labels-idx1-ubyte.gz')

成功下载  train-images-idx3-ubyte.gz 9912422 bytes.
成功下载  train-labels-idx1-ubyte.gz 28881 bytes.
成功下载  t10k-images-idx3-ubyte.gz 1648877 bytes.
成功下载  t10k-labels-idx1-ubyte.gz 4542 bytes.


In [7]:
# 查看是否下载成功

!ls /content/gdrive/My\ Drive/mylab

t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


# 3. 加载数据
## 加载图像

In [0]:
def extract_data(filename, num_images):
  """把图片加载到4维张量[image index, y, x, channels].

  像素值[0, 255]被调整到[-0.5, 0.5].
  """
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
    bytestream.read(16)  #每个像素存储在文件中的大小为16bits
    buf = bytestream.read(IMAGE_SIZE * IMAGE_SIZE * num_images * NUM_CHANNELS)
    data = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.float32)
    
    # 像素值[0, 255]被调整到[-0.5, 0.5]
    data = (data - (PIXEL_DEPTH / 2.0)) / PIXEL_DEPTH
    
    # reshape成4维张量[image index, y, x, channels]
    data = data.reshape(num_images, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)
    return data

## 加载标签

In [0]:
def extract_labels(filename, num_images):
  """把标签加载到int64型的标签向量."""
  
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
    bytestream.read(8)     #每个标签存储在文件中的大小为8bits
    buf = bytestream.read(1 * num_images)
    labels = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.int64)
  return labels

In [10]:
# 加载数据到numpy数组

train_data = extract_data(train_data_filename, 60000)
train_labels = extract_labels(train_labels_filename, 60000)
test_data = extract_data(test_data_filename, 10000)
test_labels = extract_labels(test_labels_filename, 10000)

Extracting /content/gdrive/My Drive/mylab/train-images-idx3-ubyte.gz
Extracting /content/gdrive/My Drive/mylab/train-labels-idx1-ubyte.gz
Extracting /content/gdrive/My Drive/mylab/t10k-images-idx3-ubyte.gz
Extracting /content/gdrive/My Drive/mylab/t10k-labels-idx1-ubyte.gz


# 4. 生成验证集

In [0]:
validation_data = train_data[:VALIDATION_SIZE, ...]
validation_labels = train_labels[:VALIDATION_SIZE]
train_data = train_data[VALIDATION_SIZE:, ...]
train_labels = train_labels[VALIDATION_SIZE:]
num_epochs = NUM_EPOCHS

In [12]:
# 查看训练集和验证集大小

train_size = train_labels.shape[0]
val_size = validation_labels.shape[0]

print("训练集大小： ", train_size)
print("验证集大小： ", val_size)

训练集大小：  55000
验证集大小：  5000


# 5. 构建模型计算图
## 创建输入占位符

In [0]:
train_data_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))

train_labels_node = tf.placeholder(tf.int64, shape=(BATCH_SIZE,))

eval_data = tf.placeholder(
    tf.float32,
    shape=(EVAL_BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))

## 初始化变量
调用tf.global_variables_initializer().run()时生效
![本次实验复现的卷积网络](https://pic4.zhimg.com/v2-296cbc69aa81f9ed5b56a912d2ab5c70_1200x500.jpg)

In [14]:
# 初始化卷积层1权重满足标准正态分布
# 卷积核用4维张量表示[fliter_height,filter_width,in_channels,out_channels]
conv1_weights = tf.Variable(
    tf.truncated_normal([5, 5, NUM_CHANNELS, 32],# 5x5 卷积核,  32个卷积核.
                        stddev=0.1,
                        seed=SEED, dtype=tf.float32))
# 初始化卷积层1的偏置
conv1_biases = tf.Variable(tf.zeros([32], dtype=tf.float32))

# 初始化卷积层2权重
conv2_weights = tf.Variable(tf.truncated_normal(
    [5, 5, 32, 64], stddev=0.1,
    seed=SEED, dtype=tf.float32))
# 初始化卷积层2的偏置
conv2_biases = tf.Variable(tf.constant(0.1, shape=[64], dtype=tf.float32))

# 初始化全连接层1权重
fc1_weights = tf.Variable(  # 全连接层, depth 512.
    tf.truncated_normal([IMAGE_SIZE // 4 * IMAGE_SIZE // 4 * 64, 512],
                        stddev=0.1,
                        seed=SEED,
                        dtype=tf.float32))
# 初始化全连接层1偏置
fc1_biases = tf.Variable(tf.constant(0.1, shape=[512], dtype=tf.float32))

# 初始化全连接层2权重
fc2_weights = tf.Variable(
    tf.truncated_normal([512, NUM_LABELS],
                        stddev=0.1,seed=SEED,dtype=tf.float32))
# 初始化全连接层2偏置(最终输出10维)
fc2_biases = tf.Variable(tf.constant(
    0.1, shape=[NUM_LABELS], dtype=tf.float32))

Instructions for updating:
Colocations handled automatically by placer.


## 构建模型

In [0]:
def model(data, train=False):
  """构建模型."""
  # 2维卷积用'SAME'填充 (输出feature map与输入相同) has
  # strides是一个4维数组: [image index, y, x, depth].
  
  ###################
  # 进行第一次卷积
  ###################
  conv = tf.nn.conv2d(data,
                      conv1_weights,
                      strides=[1, 1, 1, 1],
                      padding='SAME')
  
  # 偏置和ReLU非线性激活.
  relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
  
  ###################
  # 进行第一次池化(最大池化)
  ###################
  # ksize 池化窗口尺寸为2，一般第一个和最后一个参数固定为1
  # strides 池化窗口移动步幅也为2.
  pool = tf.nn.max_pool(relu,
                        ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1],
                        padding='SAME')
  
  ###################
  # 进行第二次卷积
  ###################
  conv = tf.nn.conv2d(pool,
                      conv2_weights,
                      strides=[1, 1, 1, 1],
                      padding='SAME')
  relu = tf.nn.relu(tf.nn.bias_add(conv, conv2_biases))
  
  ###################
  # 进行第二次池化(最大池化)
  ###################
  pool = tf.nn.max_pool(relu,
                        ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1],
                        padding='SAME')

  # 将feature map变换为2维矩阵，提供给全连接层
  pool_shape = pool.get_shape().as_list()  # 转化为list
  reshape = tf.reshape(pool,
      [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])
  
  # 全连接层
  hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
  
  # 训练时引入dropout防止过拟合
  if train:
    hidden = tf.nn.dropout(hidden, 0.5, seed=SEED)
  return tf.matmul(hidden, fc2_weights) + fc2_biases

## 训练与评估
### 损失计算

In [16]:
# 交叉熵损失
logits = model(train_data_node, True)
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=train_labels_node, logits=logits))

# 全连接层的L2正则化损失
regularizers = (tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc1_biases) +
                tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc2_biases))
# 求得最后损失
loss += 5e-4 * regularizers

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### 优化器优化

In [0]:
# 优化器（用于参数更新）
# 设置一个每批增加一次的变量，来控制学习率衰减
batch = tf.Variable(0, dtype=tf.float32)

# 每个epoch衰减一次，使用从0.01开始的指数衰减
learning_rate = tf.train.exponential_decay(
  0.01,                # 初始学习率.
  batch * BATCH_SIZE,  # 当前索引.
  train_size,          # 衰减步数.
  0.95,                # 衰减率.
  staircase=True)

In [0]:
# 用momentum优化器优化
optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=batch)

In [0]:
  # 预测当前的训练批
  train_prediction = tf.nn.softmax(logits)

In [0]:
# 预测验证批
eval_prediction = tf.nn.softmax(model(eval_data))

In [0]:
def eval_in_batches(data, sess):
  """预测所有."""
  size = data.shape[0]
  if size < EVAL_BATCH_SIZE:
    raise ValueError("batch size for evals larger than dataset: %d" % size)
  predictions = numpy.ndarray(shape=(size, NUM_LABELS), dtype=numpy.float32)
  for begin in xrange(0, size, EVAL_BATCH_SIZE):
    end = begin + EVAL_BATCH_SIZE
    if end <= size:
      predictions[begin:end, :] = sess.run(
          eval_prediction,
          feed_dict={eval_data: data[begin:end, ...]})
    else:
      batch_predictions = sess.run(
          eval_prediction,
          feed_dict={eval_data: data[-EVAL_BATCH_SIZE:, ...]})
      predictions[begin:, :] = batch_predictions[begin - size:, :]
  return predictions

# 创建会话

In [0]:
# 定义错误率函数
def error_rate(predictions, labels):
  return 100.0 - (
      100.0 *
      numpy.sum(numpy.argmax(predictions, 1) == labels) /
      predictions.shape[0])

In [23]:
# 创建会话进行训练
start_time = time.time()
with tf.Session() as sess:
  # 初始化可训练变量
  tf.global_variables_initializer().run()
  print('初始化完成！')
  
  # 训练循环开始
  for step in xrange(int(num_epochs * train_size) // BATCH_SIZE):
    
    # 计算当前批的偏置
    offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
    batch_data = train_data[offset:(offset + BATCH_SIZE), ...]
    batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
    
    # 将数据喂入计算图
    feed_dict = {train_data_node: batch_data,
                 train_labels_node: batch_labels}
    
    # 运行优化器更新参数
    sess.run(optimizer, feed_dict=feed_dict)
    
    # 每隔100步打印相关信息
    if step % EVAL_FREQUENCY == 0:
      # fetch some extra nodes' data
      l, lr, predictions = sess.run([loss, learning_rate, train_prediction],
                                    feed_dict=feed_dict)
      elapsed_time = time.time() - start_time
      start_time = time.time()
      print('Step %d (epoch %.2f), %.1f ms' %
            (step, float(step) * BATCH_SIZE / train_size,
             1000 * elapsed_time / EVAL_FREQUENCY))
      print('Minibatch loss: %.3f, learning rate: %.6f' % (l, lr))
      print('Minibatch error: %.1f%%' % error_rate(predictions, batch_labels))
      print('Validation error: %.1f%%' % error_rate(
          eval_in_batches(validation_data, sess), validation_labels))
      sys.stdout.flush()
      
  # 打印最后的结果
  test_error = error_rate(eval_in_batches(test_data, sess), test_labels)
  print('Test error: %.1f%%' % test_error)

初始化完成！
Step 0 (epoch 0.00), 7.5 ms
Minibatch loss: 8.334, learning rate: 0.010000
Minibatch error: 85.9%
Validation error: 84.6%
Step 100 (epoch 0.12), 212.7 ms
Minibatch loss: 3.238, learning rate: 0.010000
Minibatch error: 4.7%
Validation error: 7.3%
Step 200 (epoch 0.23), 232.8 ms
Minibatch loss: 3.352, learning rate: 0.010000
Minibatch error: 10.9%
Validation error: 4.2%
Step 300 (epoch 0.35), 224.3 ms
Minibatch loss: 3.139, learning rate: 0.010000
Minibatch error: 3.1%
Validation error: 3.0%
Step 400 (epoch 0.47), 239.2 ms
Minibatch loss: 3.212, learning rate: 0.010000
Minibatch error: 9.4%
Validation error: 2.9%
Step 500 (epoch 0.58), 291.4 ms
Minibatch loss: 3.177, learning rate: 0.010000
Minibatch error: 4.7%
Validation error: 2.5%
Step 600 (epoch 0.70), 395.9 ms
Minibatch loss: 3.114, learning rate: 0.010000
Minibatch error: 3.1%
Validation error: 2.1%
Step 700 (epoch 0.81), 384.0 ms
Minibatch loss: 2.976, learning rate: 0.010000
Minibatch error: 1.6%
Validation error: 2.2%
St